In [3]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout


In [4]:
# Load labels
labels = pd.read_csv("level_02/train_data_labels.csv", header=None).values.squeeze()

# Load training images
train_image_paths = [f"level_02/train_data/field{i:03d}.png" for i in range(500)]
train_images = [Image.open(path).convert('RGB') for path in train_image_paths]
train_data = np.array([np.asarray(image) for image in train_images]) / 255.0  # Normalize to [0, 1]




In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse')


2023-10-20 16:27:02.188710: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 16:27:02.189016: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [6]:
model.fit(train_data, labels, epochs=20, batch_size=20, validation_split=0.1)


2023-10-20 16:27:04.553645: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20


2023-10-20 16:27:04.769434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 6s 151ms/step - loss: 49.6896 - val_loss: 2.7390


2023-10-20 16:27:09.973035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/20
23/23 [==============================] - 3s 127ms/step - loss: 1.9074 - val_loss: 0.7684
Epoch 3/20
23/23 [==============================] - 3s 126ms/step - loss: 0.3834 - val_loss: 0.1877
Epoch 4/20
23/23 [==============================] - 3s 132ms/step - loss: 0.2457 - val_loss: 0.1159
Epoch 5/20
23/23 [==============================] - 3s 128ms/step - loss: 0.1196 - val_loss: 0.0804
Epoch 6/20
23/23 [==============================] - 3s 126ms/step - loss: 0.0818 - val_loss: 0.0533
Epoch 7/20
23/23 [==============================] - 3s 128ms/step - loss: 0.0580 - val_loss: 0.0473
Epoch 8/20
23/23 [==============================] - 3s 127ms/step - loss: 0.0628 - val_loss: 0.0686
Epoch 9/20
23/23 [==============================] - 3s 126ms/step - loss: 0.0504 - val_loss: 0.0427
Epoch 10/20
23/23 [==============================] - 3s 128ms/step - loss: 0.0326 - val_loss: 0.0439
Epoch 11/20
23/23 [==============================] - 3s 127ms/step - loss: 0.0315 - val_loss: 0.053

In [8]:
test_image_paths = [f"level_02/test_data/field{i:03d}.png" for i in range(1000)]
test_images = [Image.open(path).convert('RGB') for path in test_image_paths]
test_data = np.array([np.asarray(image) for image in test_images]) / 255.0  # Normalize to [0, 1]


In [9]:
predictions = model.predict(test_data).squeeze()


2023-10-20 16:28:50.925716: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [10]:
rounded_predictions = np.round(predictions).clip(0, 5)


In [11]:
np.savetxt("predictions.csv", rounded_predictions, delimiter=",", fmt='%d')
